In [ ]:
!pip install transformers
import transformers
!pip install transformers[sentencepiece]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


Με το wget αποθηκευουμε το αρχειο απο το repository στο github

In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json

--2024-01-28 11:24:25--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7460 (7.3K) [text/plain]
Saving to: ‘Dataset.json’

Dataset.json        100%[===================>]   7.29K  --.-KB/s    in 0s      

2024-01-28 11:24:25 (32.0 MB/s) - ‘Dataset.json’ saved [7460/7460]



Κάνουμε import την συνάρτηση load dataset για να φορτώσουμε τα δεδομένα απο το data set που τραβήξαμε απο το git hub.

In [ ]:
from datasets import load_dataset

Φορτώνει ενα dataset απο json αρχείο απο το πεδίο documents και τα αποθηκεύει στην μεταβλητή my_dataset.



In [ ]:
my_dataset = load_dataset("json", data_files="Dataset.json", field="documents")

Generating train split: 0 examples [00:00, ? examples/s]

χρησιμοποιούμε την εντολη print για να δούμε το περιεχόμενο του data set


In [ ]:
print(my_dataset)

DatasetDict({
    train: Dataset({
        features: ['ID', 'SUMMARIZATION', 'TEXT'],
        num_rows: 3
    })
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [ ]:
checkpoint ="facebook/bart-large-cnn"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
#how to test the tokenizer 
inputs = tokenizer("I loved reading the Hunger Games!")
print(inputs)
tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids)
print(tokens)

<font color='red'> !!!! Ο ΠΑΡΑΚΑΤΩ ΚΩΔΙΚΑΣ ΦΑΙΝΕΤΑΙ ΝΑ ΤΡΕΧΕΙ ΣΤΟ COLAB ΑΛΛΑ ΟΤΑΝ ΒΑΖΩ print("Model Inputs:", model_inputs) ΚΑΤΩ ΑΠΟ ΤΟ MODEL_INPUTS ΔΕΝ ΒΓΑΖΕΙ ΚΑΠΟΙΟ ΑΠΟΤΕΛΕΣΜΑ ΚΑΙ ΜΕ ΠΡΟΒΛΗΜΑΤΙΖΕΙ ΑΥΤΟ </font>

In [ ]:
max_input_length = 1042
max_target_length = 842

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["TEXT"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["SUMMARIZATION"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs